In [37]:
import numpy as np
import pandas as pd
import random as rd

Structure of overall applications.

Get data ready and split into two groups. Group 1 is prop on the impromptu motion on Day 1 and Group 2 is opp on the prepped motion of Day 1. On Day 2, Group 1 is opp on the impromptu and Group 2 is Prop on the impromptu. The alternation continues on Day 3 and Day 4.

Then the same process is applied for each day. First, pairs are formed between countries within groups. The pairs from both groups get paired into 2x2 blocks. 

Day 1 to 4, with the sides that groups take alternating
Pairs (for both groups)
Blocks (update penalty scores)


In [38]:
rankings = pd.read_csv('WSDCrankings.csv', sep=';')
rankings['penalty'] = pd.Series(np.repeat(0,62), index=rankings.index) # Create penalty column

In [39]:
rankings.count()

nation      62
strength    62
penalty     62
dtype: int64

In [40]:
countries = rankings['nation'].as_matrix() # Save countries as array for use later.

In [41]:
group1 = countries[range(1,61,2)]
group2 = countries[range(2,62,2)]

In [42]:
group1

array(['South Africa', 'England', 'Ireland', 'Greece', 'Malaysia',
       'Netherlands', 'Peru', 'United Arab Emirates', 'Croatia', 'Estonia',
       'Scotland', 'United States', 'Philippines', 'Denmark', 'China',
       'Israel', 'Czech Republic', 'Slovakia', 'Argentina', 'Chile',
       'Turkey', 'Lithuania', 'Sweden', 'Taiwan', 'Macau', 'Bahrain',
       'Kuwait', 'Japan', 'Nigeria', 'Rwanda'], dtype=object)

In [43]:
averageStrength = rankings['strength'].mean()
averageStrength

0.5682548387096773

In [44]:
rankings.loc[rankings['nation'].isin(group1)] # Select group of countries to start making pairs.

,nation,strength,penalty
1,South Africa,0.0859,0
3,England,0.1166,0
5,Ireland,0.1369,0
7,Greece,0.2196,0
9,Malaysia,0.2307,0
11,Netherlands,0.2769,0
13,Peru,0.2935,0
15,United Arab Emirates,0.3881,0
17,Croatia,0.3962,0
19,Estonia,0.4125,0


In [59]:
def generatePairs(group):

    # Pair countries based on two criteria: strength and penalty. 
    # Iterate through countries in the group.
    # Take the sum of the difference between the strength and penalty for each combination.
    # Choose partner with lowest sum.
    # Put paired countries on list that are no longer available for partnering.
    # Repeat.
    
    # Create a table to store all the matches in.
    pairTable = pd.DataFrame(columns=('nation', 'pairno','groupno'))
    countrieslist = countries.tolist() # These are all unmatched countries
    pairCount = 0
    
    # If the group I am matching is group 2, I remove group 1 countries.
    if group is group2: 
        pairCount += 100
        groupno = 2
        for countryRemove in group1: 
            countrieslist.remove(countryRemove)
    else:
        groupno = 1
        for countryRemove in group2:
            countrieslist.remove(countryRemove)

    for countryOne in group:

        if countryOne not in countrieslist: # If country was already matched, can't match again.
            pass

        else:
            countrieslist.remove(countryOne) # Remove now, so it is not matched with self and is not matched again.
            pairCount += 1 # Get ready to match a pair with this number

            countryOneStrength = rankings['strength'].loc[rankings['nation'] == countryOne]
            bestMatchStrength = 2 * averageStrength - countryOneStrength

            # Filter the table for countries still requiring a matching
            matchableCountries = rankings.loc[rankings['nation'].isin(countrieslist)]

            bestMatchedStrength = min(matchableCountries['strength'].as_matrix(), key=lambda x:abs(x-bestMatchStrength.values[0]))
            bestMatchedCountries = matchableCountries['nation'].loc[matchableCountries['strength'] == bestMatchedStrength].values
            bestMatchedCountry = bestMatchedCountries[rd.randint(0,len(bestMatchedCountries) - 1)]
            countrieslist.remove(bestMatchedCountry)
            
            newPairing = pd.DataFrame({'nation': [countryOne, bestMatchedCountry],
                                       'pairno': [pairCount, pairCount],
                                        'groupno': [groupno, groupno]})
            pairTable = pd.concat([pairTable, newPairing])
            
    return(pairTable)

In [107]:
def generateBlocks(pairTable):
    attributesTable = pd.DataFrame(columns=('pairno', 'strength','penalty', 'groupno'))
    pairNumbers = np.unique(pairTable['pairno'].values)
    
    for pairNumber in pairNumbers:
        pairCountries = pairTable['nation'].loc[pairTable['pairno'] == pairNumber].values
        meanStrength = np.mean(rankings['strength'].loc[rankings['nation'].isin(pairCountries)])
        meanPenalty = np.mean(rankings['penalty'].loc[rankings['nation'].isin(pairCountries)])
        groupNo = pairTable['groupno'].loc[pairTable['pairno'] == pairNumber].values[0]
        
        pairAttributes = pd.DataFrame({'pairno': [pairNumber],
                                   'strength': [meanStrength],
                                   'groupno': [groupNo],
                                  'penalty': [meanPenalty]})
        attributesTable = pd.concat([attributesTable, pairAttributes])
    
    # Let's make some blocks!
    unmatchedPairs = pairNumbers.tolist() # List of all pair numbers, because all remain to be matched.
    blockTable = pd.DataFrame(columns=('pairno', 'blockno','penalty'))
    blockCount = 0
    
    for pairNumber in pairNumbers:
        
        if pairNumber not in unmatchedPairs:
            pass
        
        else:
            # Get data necessary to sort legal matches.
            groupNo = attributesTable['groupno'].loc[attributesTable['pairno'] == pairNumber].values[0]
            unmatchedPairs.remove(pairNumber) # Remove so pair doesn't match itself.
            blockCount += 1
        
            matchablePairs = attributesTable.loc[attributesTable['groupno'] != groupNo]
            matchablePairs = matchablePairs.loc[matchablePairs['pairno'].isin(unmatchedPairs)]
        
            pairStrength = attributesTable['strength'].loc[attributesTable['pairno'] == pairNumber].values[0]
            bestMatchedStrength = min(matchablePairs['strength'].as_matrix(), key=lambda x:abs(x-pairStrength))
            bestMatchedPair = matchablePairs['pairno'].loc[matchablePairs['strength'] == bestMatchedStrength].values[0]
            unmatchedPairs.remove(bestMatchedPair)
            
            penaltyPair = bestMatchedStrength - averageStrength
            penaltyMatchedPair = pairStrength - averageStrength  
            
            newBlock = pd.DataFrame({'pairno': [pairNumber, bestMatchedPair],
                                       'blockno': [blockCount, blockCount],
                                    'penalty': [penaltyPair, penaltyMatchedPair]})
            blockTable = pd.concat([blockTable, newBlock])
    
    return blockTable

In [123]:
def generateDraw(blockTable):
    
    # The end result of this function is two tables: round 1 and round 2
    # Group 1 countries are prop in the first round


IndentationError: expected an indented block (<ipython-input-123-696a005727a6>, line 4)

In [ ]:
def updatePenalties(blockTable):
    
    # Don't forget to sort by penalty at the end!

In [108]:
pairTableGroup1 = generatePairs(group1)
pairTableGroup2 = generatePairs(group2)
pairTable = pd.concat([pairTableGroup1, pairTableGroup2])

In [111]:
blockTable = generateBlocks(pairTable)

In [120]:
generateDraw(blockTable)
updatePenalties(blockTable)

NameError: name 'generateDraw' is not defined

In [119]:
blockTable.sort_values('penalty')

,blockno,pairno,penalty
1,16,115,-0.119555
1,14,101,-0.035855
0,13,13,-0.033805
1,13,116,-0.027955
0,1,1,-0.026405
1,1,113,-0.025305
0,14,14,-0.020505
0,11,11,-0.017605
1,11,112,-0.012655
1,2,105,-0.009955
